<a href="https://colab.research.google.com/github/graham-fletcher-athome/chs/blob/main/pgn_to_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install python-chess
from google.colab import drive
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148499 sha256=3eceb8368c85f2efbe851feb31932f6750152917151c44cd95c222a08c69d639
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess
Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/chs/* .

In [ ]:
# prompt: convert fen to nn inputs like lc0 does. Add input planes for castling rights to king and queen side. Dont add a plane for side to move, invert the board before calculating the planes if it is black to move. Include a plane for en passent. Use a plane size of 10x10 with the 8x8 board in the middle. Add an extra plane that defines the position of the board

import chess
import numpy as np

def fen_to_nn_input(fen):
    """Converts a FEN string to NN input like LC0 does.

    Args:
        fen: The FEN string to convert.

    Returns:
        A NumPy array representing the NN input.
    """

    board = chess.Board(fen)
    planes = np.zeros((18, 10, 10), dtype=np.float32)

    # Invert the board if it's black to move
    if board.turn == chess.BLACK:
        board = board.mirror()

    piece_map = {
        chess.PAWN: 0,
        chess.KNIGHT: 1,
        chess.BISHOP: 2,
        chess.ROOK: 3,
        chess.QUEEN: 4,
        chess.KING: 5,
    }

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            color = int(piece.color)
            piece_type = piece_map[piece.piece_type]
            row, col = divmod(square, 8)
            planes[color * 6 + piece_type, row + 1, col + 1] = 1

    # Castling rights
    if board.has_kingside_castling_rights(chess.WHITE):
        planes[12, :, :] = 1
    if board.has_queenside_castling_rights(chess.WHITE):
        planes[13, :, :] = 1
    if board.has_kingside_castling_rights(chess.BLACK):
        planes[14, :, :] = 1
    if board.has_queenside_castling_rights(chess.BLACK):
        planes[15, :, :] = 1


    # En Passant
    if board.ep_square is not None:
        row, col = divmod(board.ep_square, 8)
        planes[16, row + 1, col + 1] = 1

    # Board Position Plane
    for row in range(8):
        for col in range(8):
          planes[17, row + 1, col + 1] = 1
    return planes

# Example usage
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
nn_input = fen_to_nn_input(fen)


In [ ]:
def pov(x):
  if x == 1:
    return -1
  return 1

In [39]:
from os import POSIX_FADV_WILLNEED
# prompt: Generate a dataset for training a NN. The inputs vector is "For all positions in all games in all the pgn files call fen_to_nn_input". The output vector is the evaluation for the position from the pgn.  Only write the first 100 lines

import chess.pgn
import io

def generate_dataset(pgn_files):
    """Generates a dataset for training a NN.

    Args:
        pgn_files: List of PGN file paths.

    Returns:
        A tuple of NumPy arrays: (inputs, outputs) where inputs are
        NN inputs and outputs are the evaluation of the position.
    """
    inputs = []
    outputs = []
    for pgn_file in pgn_files:
        with open(pgn_file, 'r') as f:
            while True:
                game = chess.pgn.read_game(f)
                if game is None:
                    break
                board = game.board()
                p_win = 0.52
                for m in game.mainline():
                    inputs.append(fen_to_nn_input(board.fen()).flatten())
                    outputs.append(p_win)

                    board.push(m.move)
                    if m.eval() is not None:
                      p_win = m.eval().wdl().relative.expectation()
                    else:
                      p_win = 1-p_win





    return np.array(inputs), np.array(outputs)


# Get a list of all pgn files in a directory
import glob
pgn_files = glob.glob("*.pgn")


# Generate dataset
inputs, outputs = generate_dataset(pgn_files)
np.savetxt("inputs.csv", inputs, delimiter=",")
np.savetxt("outputs.csv", outputs, delimiter=",")
